# Advent of code - Day10

https://adventofcode.com/2023/day/10

In [6]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

import re

# Part 1

In [7]:
def read_matrix_from_file(file_path):
    with open(file_path, 'r') as file:
        input_text = file.read()
        lines = input_text.strip().split('\n')
        matrix = [list(line) for line in lines]
        numpy_matrix = np.array(matrix)
        return numpy_matrix

field = read_matrix_from_file("day10_input.txt")

def possible_loops(start_x, start_y, field):
    
    possible_directions = []
    current = field[start_x, start_y]
    up = field[start_x-1, start_y] if start_x-1>=0 else "."
    down = field[start_x+1, start_y] if start_x+1<field.shape[0] else "."
    left = field[start_x, start_y-1]  if start_y-1>=0 else "."
    right = field[start_x, start_y+1] if start_y+1<field.shape[1] else "."
    # print("-- Current", current, "up", up, "down", down, "left", left, "right", right)

    if current == "S":
        allowed_up, allowed_down, allowed_right, allowed_left = ["|", "F", "7"], ["|", "J", "L"], ["-", "J", "7"], ["-", "F", "L"]
    if current == "-":
        allowed_up, allowed_down, allowed_right, allowed_left = [""], [""], ["-", "J", "7"], ["-", "F", "L"]
    if current == "|":
        allowed_up, allowed_down, allowed_right, allowed_left = ["|", "F", "7"], ["|", "J", "L"], [""], [""]
    if current == "J":
        allowed_up, allowed_down, allowed_right, allowed_left = ["|", "F", "7"], [""], [""], ["-", "F", "L"]
    if current == "7":
        allowed_up, allowed_down, allowed_right, allowed_left = [""], ["|", "J", "L"], [""], ["-", "F", "L"]
    if current == "F":
        allowed_up, allowed_down, allowed_right, allowed_left = [""], ["|", "J", "L"], ["-", "J", "7"], [""]
    if current == "L":               
        allowed_up, allowed_down, allowed_right, allowed_left = ["|", "F", "7"], [""], ["-", "J", "7"], [""]

    if up in allowed_up:
        # print("-- possible up", up)
        possible_directions+=[[start_x-1, start_y]]
    if down in allowed_down:
        # print("-- possible down", down)
        possible_directions+=[[start_x+1, start_y]]
    if right in allowed_right:
        # print("-- possible right", right)
        possible_directions+=[[start_x, start_y+1]]
    if left in allowed_left:
        # print("-- possible left", left)
        possible_directions+=[[start_x, start_y-1]]

    return possible_directions

start_x = int(np.where(field == "S")[0])
start_y = int(np.where(field == "S")[1])

loop_positions = [[start_x, start_y]]
additional_positions = loop_positions
# display(field)

i = 0
dist_counter = 0
while len(additional_positions)!=0:
    # print("Iteration", i, len(additional_positions))
    for position in additional_positions:
        start_x, start_y = position[0], position[1]
        # print("Checking", field[start_x, start_y], "pos", start_x, start_y)
        additional_positions = [x for x in possible_loops(start_x, start_y, field) if x not in loop_positions]
        loop_positions += additional_positions
        # print(" == final add pos", additional_positions)
    i+=1

print("longest distance", int(len(loop_positions)/2))

longest distance 6690


# Part 2

In [9]:
# Display only the loop
result_matrix = np.full_like(field, ".", dtype=np.object)
for pos in loop_positions:
    result_matrix[tuple(pos)] = field[tuple(pos)]

def replace_S(matrix):
    start_x = int(np.where(field == "S")[0])
    start_y = int(np.where(field == "S")[1])
    current = matrix[start_x, start_y]
    up = matrix[start_x-1, start_y] if start_x-1>=0 else "."
    down = matrix[start_x+1, start_y] if start_x+1<field.shape[0] else "."
    left = matrix[start_x, start_y-1]  if start_y-1>=0 else "."
    right = matrix[start_x, start_y+1] if start_y+1<field.shape[1] else "."

    if down in ["|", "L", "J"] and right in ["-", "7", "J"]:
        replace_value = "F"
    if down in ["|", "L", "J"] and left in ["-", "F", "L"]:
        replace_value = "7"
    if up in ["|", "7", "F"] and right in ["-", "F", "L"]:
        replace_value = "L"
    if up in ["|", "7", "F"] and left in ["-", "7", "J"]:
        replace_value = "J"

    matrix[start_x, start_y]= replace_value
    return matrix

replace_S(result_matrix)

#Simply count all |LJ (or alternatively |F7, same result) to the right of each point
characters_to_remap = ['|', 'L', 'J']
remapped_value = '@'
masks = [result_matrix == char for char in characters_to_remap]
combined_mask = np.logical_or.reduce(masks)
remapped_matrix = np.where(combined_mask, remapped_value, result_matrix)

# display(remapped_matrix)

count_internal = 0
for x in range(0,remapped_matrix.shape[0]):
    for y in range(0, remapped_matrix.shape[1]):
        if [x, y] not in loop_positions:

            sum_of_vertical = sum(element.count("@") for element in remapped_matrix[x][y+1:])
            if sum_of_vertical>0 and sum_of_vertical % 2 !=0:
                # print("internal")
                count_internal+=1
            else:
                # print("external")
                pass
count_internal

C:\Users\incar\AppData\Local\Temp\ipykernel_30644\2541965929.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  result_matrix = np.full_like(field, ".", dtype=np.object)


525